In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


class Dataset:
    def __init__(self, filepath, delimiter):
        self.filepath = filepath
        self.delimiter = delimiter
        self.from_categorical = {}  # '100': 0, '010': 1, '001': 2
        self.to_categorical = {}    # 0: [1,0,0], 1: [0,1,0], 2: [0,0,1]
    
    def get_raw_data(self):
        dataset = np.loadtxt(fname=self.filepath, delimiter=self.delimiter)
        return dataset[:, :-1], dataset[:, -1]
    
    def get_data(self):
        def normalize(x):
            return (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
            
        def convert_to_categorical(hot, length):
            return np.array([0 if not i == hot else 1 for i in range(length)])
        
        x_train, labels = self.get_raw_data()
        
        categories = np.sort(np.unique(labels))   
        length = len(categories)
        
        for i in range(length):
            categorical = convert_to_categorical(i, length)
            self.from_categorical[''.join(map(str, categorical))] = categories[i]
            self.to_categorical[categories[i]] = categorical
            
        y_train = list(map(lambda label: self.to_categorical[label], labels))

        return normalize(x_train), np.array(y_train)
    
    def split_shuffle(self, x, y, ratio):
        length = len(x) 
        len_train = int(round(length* ratio))
        index = list(range(length))
        random.shuffle(index)
        x_train = [x[index[i]] for i in range(len_train)]
        y_train = [y[index[i]] for i in range(len_train)]
        x_test = [x[index[i]] for i in range(len_train, length)]
        y_test = [y[index[i]] for i in range(len_train, length)]
        
        return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
    
    
    
    def plot(self):
        x, y = self.get_raw_data()
        
        pca = PCA(n_components=2)  # 2-dimensional PCA
        x = pd.DataFrame(x)

        
        plt.xlabel('x')
        plt.ylabel('y')
        colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
        
        classes = np.unique(y)
        for i, c in enumerate(classes):
            plt.scatter(x[y == c][0], x[y == c][1], label='Class {}'.format(c), c=colors[i])

        plt.show()
    
    

    

In [ ]:
import numpy as np

class Model:
    def __init__(self):
        self.nodes = [] # 2d list
        self.pseudo_root = Node(ndim=0, node_type='pseudo', activation_func=None, index=None, neuron_threshold=None)
        self.nlayer = 0
        
    def add(self, ndim, num, activation_func, neuron_threshold):
        
        node_type = 'input' if self.nlayer == 0 else 'output'
        newNodes = [Node(ndim, node_type, activation_func, index+1, neuron_threshold) for index in range(num)]
        
        # connect newNodes to last layer
        layer = [] if self.nlayer == 0 else self.nodes[self.nlayer - 1]
        for node in newNodes:
            node.input_nodes = layer
        
        # connect last layer to newNodes
        node_type = 'input' if self.nlayer == 1 else 'hidden'
        for node in layer:
            node.output_nodes = newNodes
            node.type = node_type
            
        # modify pseudo root
        self.pseudo_root.input_nodes = newNodes
            
        self.nodes.append(newNodes)
        self.nlayer += 1
        
    def train(self, x, y_hat, learning_rate):
        
        output_layer = self.nodes[self.nlayer - 1]
        Model.compute(self.pseudo_root, x)
        Model.update(self.pseudo_root, x, y_hat, learning_rate)
        
    def predict(self, x_test):
        def activate(vec):
            one_hot = [0] * len(vec)
            one_hot[np.argmax(vec)] = 1
            return one_hot
    
        output_layer = self.nodes[self.nlayer - 1]
        prediction = []
        for x in x_test:
            Model.compute(self.pseudo_root, x)
            prediction.append(activate([node.value for node in output_layer]))
            
        return np.array(prediction)
    
    @staticmethod
    def compute(root, x):
        def postorder(node):
            for input_node in node.input_nodes:
                if not input_node._id in visited:
                    postorder(input_node)
            node.compute(x)   
            visited.append(node._id)
        
        visited = []
        postorder(root)
    
    @staticmethod
    def update(output_node, x, y_hat, learning_rate):
        def preorder(root):
            visited = []
            queue = [node for node in root.input_nodes]
            while queue != []:
                front = queue.pop(0)
                
                queue += [node for node in front.input_nodes if not node._id in visited]
                front.update_weight(x=x, y_hat=y_hat, learning_rate=learning_rate)
                
                visited += [node._id for node in front.input_nodes]
        
        preorder(output_node)

In [ ]:
ID = 0
class Node:
    def __init__(self, ndim, node_type, activation_func, index, neuron_threshold):
        global ID
        self._id = ID
        self.index = index  # use for update_weight(...), start from 1, 0 is for neuron_threshold's weight
        self.type = node_type # ['input', 'hidden', 'output', 'pseudo']
        self.activation_func = activation_func
        self.neuron_threshold = neuron_threshold
        self.weight = np.random.random((ndim+1))
        
        self.value = 0
        self.gradient = 0
        self.input_nodes = []
        self.output_nodes = []
        ID += 1
    
    def get_input(self, values):
        return np.concatenate((np.array([self.neuron_threshold]), values), axis=0)
    
    def compute(self, x=[]):
        input_vec = []
        if self.type == 'pseudo':
            return
        
        if self.type == 'input':
            input_vec = self.get_input(x)
        else: # 'hidden', 'output'
            input_vec = self.get_input([node.value for node in self.input_nodes])
        
        self.value = self.activation_func(np.dot(self.weight, input_vec))
    
    def update_weight(self, x, learning_rate, y_hat):
        def get_gradient(y_hat):
            if self.type == 'output':
                return (y_hat[self.index-1] - self.value) * self.value * (1 - self.value)
            else: # 'input', 'hidden'
                output_grad = np.array([node.gradient for node in self.output_nodes])
                output_weight = np.array([node.weight[self.index] for node in self.output_nodes])
                return np.dot(output_grad, output_weight) * self.value * (1 - self.value)
                
        self.gradient = get_gradient(y_hat)
        if self.type == 'input':
            self.weight += learning_rate * self.gradient * self.get_input(x)
        else:
            self.weight += learning_rate * self.gradient * self.get_input([node.value for node in self.input_nodes])
        

In [ ]:
def get_train_data(filepath):
    dataset = np.loadtxt(fname=filepath, delimiter=delimiter)
    threshold = np.array([[neuron_threshold]] * len(dataset))
    return dataset[:, :-1], dataset[:, -1]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def plot_train(x_train, y_train):    
    plt.xlabel('x')
    plt.ylabel('y')
    for i in range(len(x_train)):
        plt.plot(x_train[i, 0], x_train[i, 1], 'b*' if y_train[i]==0 else 'r*')

    plt.show()
    
def precision(prediction, y_hat):
    correct = 0
    error = 0
    for pred, y in zip(prediction, y_hat):
        correct += 1 if np.array_equal(pred, y) else 0
        error += 1 if not np.array_equal(pred, y) else 0
    
    return correct / len(y_hat)

def fit(epoch, model, x_train, y_train, learning_rate):
    for i in range(1, epoch + 1):
        for x, y_hat in zip(x_train, y_train):
            model.train(x, y_hat, learning_rate=learning_rate)
        
        print('Epoch {}: Accuracy: {}'.format(i+1, precision(model.predict(x_test), y_test)))
        
    return model

In [ ]:
epoch = 100
dataset_dir = 'dataset/'
neuron_threshold = -2
learning_rate = 0.1
dataset_files = ['IRIS.txt', 'wine.txt', 'xor.txt', 'C10D.txt', 'C3D.txt', '8OX.txt', '5CloseS1.txt', '4satellite-6.txt']

In [ ]:
dataset = Dataset(filepath=dataset_dir + dataset_files[0], delimiter=' ')
x, y = dataset.get_data()
x_train, y_train, x_test, y_test = dataset.split_shuffle(x, y, 2/3)

In [ ]:
dataset.plot()

In [ ]:
input_dim = len(x_train[0])
output_dim = len(y_train[0])

In [ ]:
model = Model()
model.add(ndim=input_dim, num=3, activation_func=sigmoid, neuron_threshold=neuron_threshold)
#     model.add(ndim=3, num=3, activation_func=sigmoid, neuron_threshold=neuron_threshold)
#     model.add(ndim=3, num=3, activation_func=sigmoid, neuron_threshold=neuron_threshold)
#     model.add(ndim=2, num=2, activation_func=sigmoid, neuron_threshold=neuron_threshold)
model.add(ndim=3, num=output_dim, activation_func=sigmoid, neuron_threshold=neuron_threshold)

In [ ]:
fit(epoch, model, x_train, y_train, learning_rate)